In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import utils, callbacks
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, LeakyReLU
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


### Create train and validation dataset using data augmentation

In [4]:

BATCH_SIZE = 8
IMG_SIZE = (224, 224)
DATASET_DIR = "augmented_dataset"

train_dataset = tf.keras.utils.image_dataset_from_directory(f"{DATASET_DIR}/train",
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,
                                                            validation_split=0.1,
                                                            subset="training",
                                                            label_mode="categorical",
                                                            seed=42)

validation_dataset = tf.keras.utils.image_dataset_from_directory(f"{DATASET_DIR}/train",
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,
                                                            validation_split=0.1,
                                                            subset="validation",
                                                            label_mode="categorical",
                                                            seed=42)



Found 78000 files belonging to 26 classes.
Using 70200 files for training.
Found 78000 files belonging to 26 classes.
Using 7800 files for validation.


### Creation of test dataset

In [5]:
test_dataset = tf.keras.utils.image_dataset_from_directory(f"pre_process_test_dataset/train",
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,

                                                            label_mode="categorical",
                                                            seed=42)
                 

Found 122 files belonging to 26 classes.


In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Pre-process input to VGG16

In [7]:
# preprocess_input = tf.keras.applications.vgg16.preprocess_input
preprocess_input = tf.keras.applications.resnet50.preprocess_input

### Create Base Model

In [8]:
IMG_SHAPE = IMG_SIZE + (3,)

# base_model = tf.keras.applications.vgg16.VGG16(
#     include_top=False,
#     weights='imagenet',
#     input_shape=IMG_SHAPE,
# )

base_model = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=IMG_SHAPE,
)

# tf.keras.applications.resnet50.ResNet50(
#     include_top=True,
#     weights='imagenet',
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     **kwargs
# )


base_model.trainable = False

94765736/94765736 [==============================] - 6s 0us/step


### Create Classification Head

In [9]:
# image_batch, label_batch = next(iter(train_dataset))
# feature_batch = base_model(image_batch)
# print(feature_batch.shape)

# flattens vectors
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# feature_batch_average = global_average_layer(feature_batch)
# print(feature_batch_average.shape)

# classification head
n_classes = 26
prediction_layer = tf.keras.Sequential([
    tf.keras.layers.Dense(4096, activation="relu"),
    tf.keras.layers.Dense(4096, activation="relu"),
    tf.keras.layers.Dense(n_classes, activation="softmax")
])

# prediction_batch = prediction_layer(feature_batch_average)
# print(prediction_batch.shape)


### Create Final Model

In [10]:

inputs = tf.keras.Input(shape=IMG_SHAPE)
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

### HyperParameters of Model

In [11]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# model.summary()

### Training

In [12]:
initial_epochs = 10

history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)


Epoch 1/10
2543/8775 [=======>......................] - ETA: 54:07 - loss: 0.4739 - accuracy: 0.8553

KeyboardInterrupt: 

### Plot Learning Curves

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Evaluate Model

In [ ]:
loss, accuracy = model.evaluate(test_dataset)

### Save Model

In [ ]:
model.save('resnet50_models/resnet')
model.save_weights('resnet50_models/resnet_weights')